<a href="https://colab.research.google.com/github/easyri/paintings/blob/master/attention2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import pandas as pd
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os, glob, cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications import InceptionResNetV2, InceptionV3

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.preprocessing import image
from keras.layers import Flatten, Input

import torch
from torch import nn
from collections import OrderedDict
import argparse
import os
from torchvision import transforms, utils, datasets, models
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import time
import random
import glob
import copy
import shutil
import sys
from matplotlib.ticker import MaxNLocator
from keras.layers import Reshape, Concatenate
from collections import defaultdict
import string
from os import listdir
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from pickle import dump
from sklearn.model_selection import train_test_split
from pickle import load
import argparse
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Input, Reshape, Concatenate

In [50]:
dataset = pd.read_csv('/content/drive/My Drive/paintings/wiki_clear_dataset.csv')

In [66]:
names = dataset["Name"]
names.describe()

count              78587
unique             64711
top       self portrait 
freq                 493
Name: Name, dtype: object

In [ ]:
names

In [62]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text


# extract descriptions for images
def load_descriptions(doc):
  mapping = dict()
  # process lines
  for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    if len(line) < 2:
      continue
    # take the first token as the image id, the rest as the description
    image_id, image_desc = tokens[0], tokens[1:]
    # remove filename from image id
    image_id = image_id.split('.')[0]
    # convert description tokens back to string
    image_desc = ' '.join(image_desc)
    # create the list if needed
    if image_id not in mapping:
      mapping[image_id] = list()
    # store description
    mapping[image_id].append(image_desc)
  return mapping


# load a pre-defined list of photo identifiers
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  # process line by line
  for line in doc.split('\n'):
    # skip empty lines
    if len(line) < 1:
      continue
    # get the image identifier
    identifier = line.split('.')[0]
    dataset.append(identifier)
  return set(dataset)

def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    # split id from description
    image_id, image_desc = tokens[0], tokens[1:]
    # skip images not in the set
    if int(image_id) in dataset:
      # create list
      if int(image_id) not in descriptions:
        descriptions[int(image_id)] = list()
      # wrap description in tokens
      desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
      # store
      descriptions[int(image_id)].append(desc)
  return descriptions

def clean_descriptions(descriptions):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, desc in descriptions.items():
      # tokenize
    desc = desc.split()
      # convert to lower case
    desc = [word.lower() for word in desc]
      # remove punctuation from each token
    desc = [w.translate(table) for w in desc]
      # remove hanging 's' and 'a'
    desc = [word for word in desc if len(word)>1]
      # remove tokens with numbers in them
    desc = [word for word in desc if word.isalpha()]
      # store as string
    desc =  ' '.join(desc)

def to_vocabulary(descriptions):
  # build a list of all description strings
  all_desc = set()
  for key in descriptions.keys():
    [all_desc.update(d.split()) for d in descriptions[key]]
  print('Vocabulary Size: %d' % len(all_desc))
  return all_desc

# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc in descriptions.items():
    lines.append(str(key) + ' ' + desc)
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()

  # convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
  lines = to_lines(descriptions)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
  lines = to_lines(descriptions)
  return max(len(d.split()) for d in lines)

# load photo features
def load_photo_features(filename, dataset):
  # load all features
  all_features = dict(enumerate(np.load(filename)))
  # filter features
  features = {k: all_features[k] for k in dataset}
  return features

In [71]:
train, test = train_test_split(names.index, random_state=0)
print('Train=%d, Test=%d' % (len(train), len(test)))

# descriptions
train_descriptions = load_clean_descriptions('/content/drive/My Drive/paintings/wiki_descr.txt', train)
test_descriptions = load_clean_descriptions('/content/drive/My Drive/paintings/wiki_descr.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))


Train=58940, Test=19647
Descriptions: train=58940, test=19647


In [73]:
train_desc = {}
test_desc = {}
for item, descr in train_descriptions.items():
  train_desc[dataset["Filename"][item]] = train_descriptions[item]
for item, descr in test_descriptions.items():
  test_desc[dataset["Filename"][item]] = test_descriptions[item]

In [74]:
train_descriptions = train_desc
test_descriptions = test_desc

In [75]:
def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

In [76]:
lines = to_lines(train_descriptions)
max_len = max_length(train_descriptions)
print(lines)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
train_seqs = tokenizer.texts_to_sequences(lines)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
train_seqs = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=max_len, padding='post')
print('Vocabulary Size: %d' % vocab_size)
print(cap_vector.shape, len(train_seqs))

['startseq acolman endseq', 'startseq chicago endseq', 'startseq gloucester a endseq', 'startseq jerome arizona endseq', 'startseq kentucky endseq', 'startseq new york endseq', 'startseq new york endseq', 'startseq new york endseq', 'startseq per endseq', 'startseq pleasures and terrors of levitation endseq', 'startseq recife olinda endseq', 'startseq rome arch of constantine endseq', 'startseq utah endseq', 'startseq vera cruz endseq', 'startseq westport endseq', 'startseq abstract composition endseq', 'startseq antibes endseq', 'startseq saman sar s illustration endseq', 'startseq number abstract painting yellow endseq', 'startseq cosmogarden endseq', 'startseq first night i endseq', 'startseq midnight spaghetti endseq', 'startseq taxi cab ii endseq', 'startseq man endseq', 'startseq maquette for flamingo endseq', 'startseq the dog endseq', 'startseq two discs endseq', 'startseq erg series endseq', 'startseq gate of hope endseq', 'startseq stargazer endseq', 'startseq abstract compos

In [77]:
# image = img_name_val[0]
# print(image)

img_to_cap_vector = {}
for name, cap in zip(list(train_descriptions.keys()), cap_vector):
   img_to_cap_vector[name] = cap
# real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[0] if i not in [0]])
# print(real_caption)

In [ ]:
img_name_train

In [90]:
img_keys = list(img_to_cap_vector.keys())
# random.shuffle(img_keys)

# разделение train на train и val

# slice_index = int(len(img_keys)*0.8)
img_name_train_keys, img_name_val_keys = train_test_split(img_keys, random_state=0)

img_name_train = []
cap_train = []
for imgt in img_name_train_keys:
  img_name_train.append(imgt)
  cap_train.append(img_to_cap_vector[imgt])

img_name_val = []
cap_val = []
for imgv in img_name_val_keys:
  img_name_val.append(imgv)
  cap_val.append(img_to_cap_vector[imgv])

In [91]:
print(len(cap_val), len(img_name_val))

14557 14557


In [80]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 1536
attention_features_shape = 64

In [113]:
print('/content/drive/MyDrive/paintings123/single_features/{}.npy'.format(img_name_train[0]))

/content/drive/MyDrive/paintings123/single_features/antoine-watteau_standing-man.jpg.npy


In [116]:
len(os.listdir('/content/drive/MyDrive/paintings123/single_features'))

45709

In [118]:
len(img_keys)

58228

In [114]:
def map_func(img_name, cap):
  img_tensor = np.load('/content/drive/MyDrive/paintings123/single_features/{}.npy'.format(img_name))
  return img_tensor, cap

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [93]:
test_dataset = tf.data.Dataset.from_tensor_slices((img_name_val, cap_val))
test_dataset = test_dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [83]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)
    # print('features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)', features.shape)
    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    # print('context_vector shape after sum == (batch_size, hidden_size)', context_vector.shape)

    return context_vector, attention_weights



In [84]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)
        

    def call(self, x):
        # print('initial cnn shape', x.shape)
        x = self.fc(x)
        # print('shape after fc == (batch_size, 64, embedding_dim)', x.shape)
        x = tf.nn.relu(x)
        return x


class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.dropout = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)
    self.batchnormalization = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))
    
    x= self.dropout(x)
    x= self.batchnormalization(x)

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [85]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')



In [86]:
checkpoint_path = "/content/drive/MyDrive/paintings123/checkp"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  print("Restored from {}".format(ckpt_manager.latest_checkpoint))
  ckpt.restore(ckpt_manager.latest_checkpoint)
print(start_epoch)

0


In [87]:
loss_plot = []
test_loss_plot = []

In [88]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss


@tf.function
def test_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * target.shape[0], 1)

  features = encoder(img_tensor)

  for i in range(1, target.shape[1]):
          # passing the features through the decoder
      predictions, hidden, _ = decoder(dec_input, features, hidden)

      loss += loss_function(target[:, i], predictions)

          # using teacher forcing
      dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))


  return loss, total_loss

In [115]:


EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0
    for batch, (img_tensor, target) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    total_loss_test = 0
    for batch, (img_tensor, target) in enumerate(test_dataset):
        batch_loss, t_loss = test_step(img_tensor, target)
        total_loss_test += t_loss
    test_loss_plot.append(total_loss_test / num_steps)
    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f} Test Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps, total_loss_test / num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
np.save('/content/drive/MyDrive/paintings123/plots/train_loss.npy', loss_plot)    
np.save('/content/drive/MyDrive/paintings123/plots/test_loss.npy', test_loss_plot)

UnknownError: ignored

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = preprocess_input(img)
    return img, image_path
  

def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val[0])

    dec_input = tf.expand_dims([tokenizer.word_index['startseq']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == 'endseq':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()


In [ ]:
print(img_name_val)

[['viktor-vasnetsov_pharmacist-1871.jpg'], ['vincent-van-gogh_peasant-woman-digging-1885.jpg'], ['boris-kustodiev_bridge-astrakhan-1918.jpg'], ['thomas-eakins_an-arcadian.jpg'], ['nikolay-bogdanov-belsky_visit-of-the-unhealthy-teacher.jpg'], ['vasily-vereshchagin_the-three-main-gods-in-a-chingacheling-buddhist-monastery-in-sikkim-1875.jpg'], ['vincent-van-gogh_orphans-1882.jpg'], ['thomas-eakins_portrait-of-dr-edward-anthony-spitzka.jpg'], ['john-singer-sargent_sir-frank-swettenham-1904.jpg'], ['gustave-courbet_the-beach-at-trouville-at-low-tide-1865.jpg'], ['thomas-eakins_nude-man-seated.jpg'], ['vincent-van-gogh_basket-of-carnations-and-zinnias-1886(1).jpg'], ['ilya-repin_under-escort-on-the-muddy-road-1876.jpg'], ['theodore-gericault_portrait-of-a-kleptomaniac-1822.jpg'], ['george-catlin_co-ee-h-jo-a-seminole-chief-1837.jpg'], ['vincent-van-gogh_snowy-landscape-with-stooping-woman-1883.jpg'], ['aleksey-savrasov_riverbank-be.jpg'], ['vasily-surikov_yermak-s-conquest-of-siberia-1895.j

In [ ]:
# captions on the validation set
from IPython.display import Image, display
for rid in range(10):
  image = img_name_val[rid]
  real_caption = ' '.join([tokenizer.index_word[i] for i in img_to_cap_vector[image[0]] if i not in [0]])
  display(Image('/content/drive/MyDrive/paintings/wikiart/test/attention_test/{}'.format(image[0]), width=300, height=300))
  image_path = '/content/drive/MyDrive/paintings/wikiart/test/attention_test/{}'.format(image[0])
  result, attention_plot = evaluate(image_path)
  print ('Real Caption:', real_caption)
  print ('Prediction Caption:', ' '.join(result[:-1]))
# plot_attention(image, result, attention_plot)

In [119]:
images = load(open('/content/drive/MyDrive/paintings123/inc_impr_arrays3.pkl', 'rb'))


In [120]:
image_model = InceptionResNetV2(include_top='False', weights='imagenet')

new_input = image_model.input
hidden_layer = image_model.layers[-3].output
print(hidden_layer.shape)

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
# image_features_extract_model.summary()

225214464/225209952 [==============================] - 9s 0us/step
(None, 8, 8, 1536)


In [ ]:



for name, image in images.items():
  image = np.expand_dims(image, axis=0)
  image = preprocess_input(image)
  batch_features = image_features_extract_model(image)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))
  path = '/content/drive/MyDrive/paintings123/single_features/{}'.format(name)
  # print(path)
  np.save(path, batch_features)